In [20]:
import pandas as pd
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, json
from urllib import request, parse
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
from time import sleep
import time, timeit
import asyncio, aiohttp
from selenium.webdriver.common.keys import Keys

In [21]:
#pickle 데이터 불러오기
with open('Gameinfo_70000_to_110000.pickle', 'rb') as f:
    match_df = pickle.load(f)
#챔피언 id/이름 목록 파일 불러오기
with open('champion_id_and_name_list.pickle','rb') as f1:
    champ_id_dict=pickle.load(f1)

In [22]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23016 entries, 0 to 0
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   gameId                 23016 non-null  object
 1   gameCreation           23016 non-null  object
 2   gameDuration           23016 non-null  object
 3   queueId                23016 non-null  object
 4   gameVersion            23016 non-null  object
 5   teams                  23016 non-null  object
 6   participants           23016 non-null  object
 7   participantIdentities  23016 non-null  object
dtypes: object(8)
memory usage: 1.6+ MB


In [23]:
match_df.index = range(len(match_df))

In [24]:
def parse_tup(summonerName, championId): #동원님 스크립트를 이용해서 함수 재조합.
    global driver, champ_id_dict
    name_parsed = parse.quote_plus(summonerName)
    champ = champ_id_dict[championId] #id -챔피언명 정보 딕셔너리로 불러와주세요. 딕셔너리 이름 다르게 했으면 이 이름으로 하셔야 합니다. 
    try:
        driver.get(f'https://www.op.gg/summoner/userName={name_parsed}')
        driver.find_element_by_xpath('//*[@id="right_gametype_soloranked"]/a').send_keys(Keys.ENTER)
        sleep(2)
        x=driver.find_element_by_xpath('//*[@id="GameAverageStatsBox-matches"]/div[1]/table/tbody/tr[2]/td[2]/div[2]')
        spans=x.find_elements_by_xpath('./span')
        recentKDA= float(spans[0].text.split(':')[0])
        recentW_R= int(spans[1].find_element_by_xpath('./span').text[:-1])
        totalW_R=int(driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[2]/div[1]/div[1]/div/div[2]/div[3]/span[2]/span[3]').text.split()[-1][:-1])
        driver.get(f'https://www.op.gg/summoner/champions/userName={name_parsed}') #url 변경으로 대체가능
        driver.find_element_by_xpath('//*[@id="champion_season"]/li[2]/a').send_keys(Keys.ENTER)
        sleep(2)
        chamW_R=0
        chamKDA=0
        matchCNT=0
        table = driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[3]/div/div/div[2]/div[1]/table/tbody')
        trs=table.find_elements_by_xpath('./tr')
        #픽 champion의 시즌 승률 검색 만약 데이터가 없으면 최근승률로 대체
        for tr in trs:
                tds = tr.find_elements_by_xpath('./td')
                if tds[2].get_attribute('data-value')==champ:
                    chamW_R=int(tds[3].find_element_by_xpath('.//span').text[:-1])
                    chamKDA=float(tds[4].get_attribute("data-value"))
                    divs=tds[3].find_elements_by_xpath('./div/div/div')
                    matchCNT=int(divs[1].text[:-1])+int(divs[3].text[:-1])
                    break
        else:
            chamW_R=recentW_R
            chamKDA=recentKDA
                
        
        return [summonerName,champ,chamW_R,chamKDA,recentW_R,recentKDA, totalW_R ,matchCNT]
    except: #추출 불가면 아무것도 return하지 않음.
        return 'err!'
    

In [25]:
data_list = []
data_list_failed = []
data_list_errored = []

In [ ]:
url_f = 'https://www.op.gg/summoner/userName={}'
#언어 설정 영어로 지정

i = 0
start_time = time.time()
for i in list(range(5000,20000,100)): #+list(range(4000, len(match_df), 100)): #900까지 포크했으니...
    #options = webdriver.ChromeOptions() #headless option  사용 - 창 띄우지 않고 실행
    #options.add_argument('headless')
    #options.add_argument('window-size=800x600')
    #options.add_argument("disable-gpu")
    driver= webdriver.Chrome() #이번엔 옵션 없애기...
    driver.get('https://op.gg/')
    driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
    sleep(2)
    lang_list = driver.find_elements_by_class_name('setting-list__item')
    for lang in lang_list:
        if "English" in lang.text:
            lang.click()
    sleep(2)
    driver.find_elements_by_class_name('setting__button')[0].click()
    sleep(2)
    for j in tqdm(range(100)): # 100단위로 처리     
    #userlist parsing
        try:
            team_blue = {'gameId':match_df.gameId[i+j], 'teamId':100 }
            team_red = {'gameId':match_df.gameId[i+j], 'teamId':200 }
            team_blue['win'] = True if match_df.teams[i+j][0]['win'] in ['Win', True, 'True'] else False
            team_red['win'] = True if match_df.teams[i+j][1]['win'] in ['Win', True, 'True'] else False
            #우선 둘이 같으면 데이터 오류로 판단하고 기각
            dat_err = False
            if team_blue['win'] == team_red['win']:
                dat_err = True
            team_blue['participants'] = []
            team_red['participants'] = []
            temp_blue =[]
            temp_red = []
            if not dat_err:
                for k in [0,5]: #blue : k=0, red : k=5
                    for l in range(5): #participantIdentities 에서 userid 가져옴,
                        p_info = match_df.participantIdentities[i+j][l+k]
                        p_name = p_info['player']['summonerName']
                        name_parsed = parse.quote_plus(p_name)
                        url = url_f.format(name_parsed)
                        html = request.urlopen(url).read().decode('utf8')
                        soup = BeautifulSoup(html, 'html.parser')
                        title = soup.title.string
                        if title[-1] == 'G': #반복 횟수 줄이기 위한 테크닉
                            break
                        elif k ==0: 
                            p_champ = match_df.participants[i+j][l]['championId']
                            tup = (p_name, p_champ)
                            temp_blue.append(tup)
                        elif k ==5: #페이지 유효
                            p_champ = match_df.participants[i+j][l+5]['championId']
                            tup = (p_name, p_champ)
                            temp_red.append(tup)


                if len(temp_blue)==5: #5명 모두 긁어올 때만
                    parsed_res =[]
                    for tup in temp_blue:
                        try: 
                            res = parse_tup(*tup)
                            parsed_res.append(res)
                        except:
                            data_list_failed.append(i+j)
                            break
                        
                    if len(parsed_res) == 5: 
                        if (not ('err!' in parsed_res)): 
                            team_blue['participants'] = parsed_res
                            data_list.append(team_blue)
                        else:
                            data_list_errored.append(i+j)

                if len(temp_red)==5: #5명 모두 긁어올 때만.
                    parsed_res =[]
                    for tup in temp_red:
                        try:
                            res = parse_tup(*tup)
                            parsed_res.append(res)
                        except:
                            data_list_failed.append(i+j)
                        
                    if len(parsed_res) == 5: 
                        if (not ('err!' in parsed_res)): 
                            team_red['participants'] = parsed_res
                            data_list.append(team_red)
                        else:
                            data_list_errored.append(i+j)
        except:
            pass
        
    with open(f'Gameinfo_with_user_70000_to_110000_{i//100+1}_m.pickle', 'wb') as pk1: #100개 긁어올 때마다...
        pickle.dump(data_list, pk1)
        print(len(data_list), len(data_list_failed)) #데이터 길이 체크.
        data_list = []
        
    
    driver.quit() #100개 모을 때마다 강제휴식 
    sleep(10)

print('end time', time.time()-start_time)
        
            


 19%|█████████████▋                                                          | 19/100 [16:12<43:23, 32.14s/it]

In [16]:
data_list_failed

[]

In [17]:
data_list_errored

[]

In [19]:
team_blue, team_red

({'gameId': 0    4751882004
  0    4751891353
  0    4751930286
  0    4751789156
  0    4751787371
          ...    
  0    4733699592
  0    4733664630
  0    4733439686
  0    4733444788
  0    4733430403
  Name: gameId, Length: 23016, dtype: object,
  'teamId': 100},
 {'gameId': 0    4751882004
  0    4751891353
  0    4751930286
  0    4751789156
  0    4751787371
          ...    
  0    4733699592
  0    4733664630
  0    4733439686
  0    4733444788
  0    4733430403
  Name: gameId, Length: 23016, dtype: object,
  'teamId': 200})